#  Guia Prático - Redes Convolucionais (CNN)
## 

Este notebook contém templates e exemplos prontos para usar em provas práticas de redes convolucionais.

## 📚 1. IMPORTS ESSENCIAIS

In [ ]:
# Imports básicos
import numpy as np
import matplotlib.pyplot as plt
import os, shutil, pathlib

# TensorFlow/Keras
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
from keras.datasets import mnist, cifar10, cifar100
from keras.applications import VGG16, ResNet50, MobileNetV2

# Para visualização
import seaborn as sns
plt.style.use('seaborn-v0_8')

## 🏗️ 2. TEMPLATES DE MODELOS CNN

In [ ]:
# 🔥 TEMPLATE 1: CNN Básica para MNIST (28x28, 10 classes)
def create_mnist_cnn():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

# 🔥 TEMPLATE 2: CNN para CIFAR-10 (32x32, 10 classes)
def create_cifar_cnn():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

# 🔥 TEMPLATE 3: CNN para imagens maiores (180x180, 2 classes - binary)
def create_binary_cnn():
    inputs = keras.Input(shape=(180, 180, 3))
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

# 🔥 TEMPLATE 4: CNN com Data Augmentation
def create_cnn_with_augmentation():
    # Data augmentation
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ])
    
    inputs = keras.Input(shape=(180, 180, 3))
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

## 📊 3. CARREGAMENTO DE DADOS

### 🔥 MNIST Dataset (28x28, 10 classes)

In [ ]:
# Carregar MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Pré-processamento
train_images = train_images.reshape(60000, 28, 28, 1).astype("float32") / 255
test_images = test_images.reshape(10000, 28, 28, 1).astype("float32") / 255

# Criar e treinar modelo
model = create_mnist_cnn()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Treinar
history = model.fit(train_images, train_labels, 
                   epochs=5, batch_size=64,
                   validation_split=0.2)

# Avaliar
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

# Visualizar algumas predições
def show_predictions(model, test_images, test_labels, num_samples=10):
    predictions = model.predict(test_images[:num_samples])
    predicted_classes = np.argmax(predictions, axis=1)
    
    plt.figure(figsize=(15, 3))
    for i in range(num_samples):
        plt.subplot(2, 5, i+1)
        plt.imshow(test_images[i].reshape(28, 28), cmap='gray')
        plt.title(f'True: {test_labels[i]}, Pred: {predicted_classes[i]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_predictions(model, test_images, test_labels)

### 🔥 CIFAR-10 Dataset (32x32, 10 classes)


In [ ]:
# Carregar CIFAR-10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Pré-processamento
train_images = train_images.astype("float32") / 255
test_images = test_images.astype("float32") / 255

# Criar e treinar modelo
model = create_cifar_cnn()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Treinar
history = model.fit(train_images, train_labels, 
                   epochs=10, batch_size=64,
                   validation_split=0.2)

# Avaliar
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

# Classes do CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

# Visualizar predições
def show_cifar_predictions(model, test_images, test_labels, num_samples=10):
    predictions = model.predict(test_images[:num_samples])
    predicted_classes = np.argmax(predictions, axis=1)
    
    plt.figure(figsize=(15, 3))
    for i in range(num_samples):
        plt.subplot(2, 5, i+1)
        plt.imshow(test_images[i])
        plt.title(f'True: {class_names[test_labels[i][0]]}\nPred: {class_names[predicted_classes[i]]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_cifar_predictions(model, test_images, test_labels)


### 🔥 Carregamento de Imagens Customizadas (Dogs vs Cats)


In [ ]:
# Função para organizar dataset de imagens customizadas
def organize_custom_dataset(original_dir, new_base_dir, train_size=1000, val_size=500, test_size=500):
    """
    Organiza imagens em subconjuntos de treino, validação e teste
    """
    def make_subset(subset_name, start_index, end_index):
        for category in ("cat", "dog"):
            dir_path = new_base_dir / subset_name / category
            os.makedirs(dir_path, exist_ok=True)
            fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
            for fname in fnames:
                src = original_dir / fname
                dst = dir_path / fname
                if src.exists():
                    shutil.copyfile(src, dst)
    
    make_subset("train", 0, train_size)
    make_subset("validation", train_size, train_size + val_size)
    make_subset("test", train_size + val_size, train_size + val_size + test_size)
    print(f"Dataset organizado em {new_base_dir}")

# Exemplo de uso (assumindo que você tem o dataset baixado)
# organize_custom_dataset(pathlib.Path("train"), pathlib.Path("cats_vs_dogs_small"))

# Carregar dataset usando image_dataset_from_directory
def load_custom_dataset(data_dir, image_size=(180, 180), batch_size=32):
    """
    Carrega dataset de imagens usando image_dataset_from_directory
    """
    train_dataset = image_dataset_from_directory(
        data_dir / "train",
        image_size=image_size,
        batch_size=batch_size,
    )
    
    validation_dataset = image_dataset_from_directory(
        data_dir / "validation",
        image_size=image_size,
        batch_size=batch_size,
    )
    
    test_dataset = image_dataset_from_directory(
        data_dir / "test",
        image_size=image_size,
        batch_size=batch_size,
    )
    
    return train_dataset, validation_dataset, test_dataset

# Exemplo de uso:
# train_dataset, validation_dataset, test_dataset = load_custom_dataset(pathlib.Path("cats_vs_dogs_small"))


## 🔄 4. DATA AUGMENTATION

In [ ]:
# 🔥 TÉCNICAS DE DATA AUGMENTATION

# 1. Data Augmentation Básico
def create_basic_augmentation():
    return keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ])

# 2. Data Augmentation Avançado
def create_advanced_augmentation():
    return keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.3),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ])

# 3. Data Augmentation para imagens médicas/científicas
def create_medical_augmentation():
    return keras.Sequential([
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(height_factor=0.05, width_factor=0.05),
        layers.RandomContrast(0.1),
    ])

# 4. Função para visualizar data augmentation
def visualize_augmentation(augmentation_layer, images, num_samples=8):
    """
    Visualiza o efeito do data augmentation em algumas imagens
    """
    augmented_images = augmentation_layer(images[:num_samples])
    
    plt.figure(figsize=(16, 4))
    for i in range(num_samples):
        plt.subplot(2, 4, i+1)
        plt.imshow(augmented_images[i].numpy())
        plt.title(f'Augmented {i+1}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Exemplo de uso:
# augmentation = create_basic_augmentation()
# visualize_augmentation(augmentation, train_images)


In [ ]:
# 🔥 CALLBACKS ESSENCIAIS PARA TREINAMENTO

# 1. Callbacks básicos
def create_basic_callbacks(model_name="model"):
    return [
        keras.callbacks.ModelCheckpoint(
            filepath=f"{model_name}.keras",
            save_best_only=True,
            monitor="val_loss",
            verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

# 2. Callbacks avançados
def create_advanced_callbacks(model_name="model", log_dir="logs"):
    return [
        keras.callbacks.ModelCheckpoint(
            filepath=f"{model_name}.keras",
            save_best_only=True,
            monitor="val_accuracy",
            mode="max",
            verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=5,
            min_lr=1e-8,
            verbose=1
        ),
        keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            write_images=True
        ),
        keras.callbacks.CSVLogger(f"{model_name}_training.log")
    ]

# 3. Função para plotar histórico de treinamento
def plot_training_history(history):
    """
    Plota gráficos de loss e accuracy durante o treinamento
    """
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# 4. Função para avaliar modelo
def evaluate_model(model, test_dataset, class_names=None):
    """
    Avalia o modelo e mostra métricas detalhadas
    """
    test_loss, test_acc = model.evaluate(test_dataset, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # Predições
    predictions = model.predict(test_dataset)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Matriz de confusão
    true_labels = []
    for images, labels in test_dataset:
        true_labels.extend(labels.numpy())
    true_labels = np.array(true_labels)
    
    from sklearn.metrics import confusion_matrix, classification_report
    
    cm = confusion_matrix(true_labels, predicted_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    if class_names:
        print("\nClassification Report:")
        print(classification_report(true_labels, predicted_classes, target_names=class_names))
    
    return test_loss, test_acc


## 🚀 6. TRANSFER LEARNING E FINE-TUNING

In [ ]:
# 🔥 TRANSFER LEARNING - FEATURE EXTRACTION

# 1. Usando VGG16 para feature extraction
def create_vgg16_feature_extractor(input_shape=(180, 180, 3), num_classes=2):
    """
    Cria modelo usando VGG16 como feature extractor
    """
    # Carregar VGG16 pré-treinado
    conv_base = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Congelar camadas convolucionais
    conv_base.trainable = False
    
    # Criar modelo completo
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.vgg16.preprocess_input(inputs)
    x = conv_base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs, outputs)

# 2. Usando ResNet50 para feature extraction
def create_resnet50_feature_extractor(input_shape=(180, 180, 3), num_classes=2):
    """
    Cria modelo usando ResNet50 como feature extractor
    """
    conv_base = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    conv_base.trainable = False
    
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = conv_base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs, outputs)

# 3. Fine-tuning de VGG16
def create_vgg16_finetuned(input_shape=(180, 180, 3), num_classes=2):
    """
    Cria modelo VGG16 com fine-tuning
    """
    conv_base = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Congelar todas as camadas exceto as últimas
    conv_base.trainable = True
    for layer in conv_base.layers[:-4]:
        layer.trainable = False
    
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.vgg16.preprocess_input(inputs)
    x = conv_base(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    # Compilar com learning rate baixo para fine-tuning
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 4. Função para extrair features de um dataset
def extract_features(model, dataset, batch_size=32):
    """
    Extrai features de um dataset usando um modelo pré-treinado
    """
    features = []
    labels = []
    
    for images, batch_labels in dataset:
        batch_features = model.predict(images, verbose=0)
        features.append(batch_features)
        labels.append(batch_labels)
    
    return np.concatenate(features), np.concatenate(labels)

## 🎯 7. EXEMPLO COMPLETO - WORKFLOW DE TREINAMENTO

In [ ]:
# 🔥 WORKFLOW COMPLETO DE TREINAMENTO

def complete_training_workflow(dataset_type="mnist"):
    """
    Workflow completo de treinamento para diferentes tipos de dataset
    """
    
    if dataset_type == "mnist":
        # 1. Carregar dados
        (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
        train_images = train_images.reshape(60000, 28, 28, 1).astype("float32") / 255
        test_images = test_images.reshape(10000, 28, 28, 1).astype("float32") / 255
        
        # 2. Criar modelo
        model = create_mnist_cnn()
        model.compile(optimizer="rmsprop",
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])
        
        # 3. Callbacks
        callbacks = create_basic_callbacks("mnist_model")
        
        # 4. Treinar
        print("🚀 Iniciando treinamento MNIST...")
        history = model.fit(train_images, train_labels,
                           epochs=10, batch_size=64,
                           validation_split=0.2,
                           callbacks=callbacks)
        
        # 5. Avaliar
        test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
        print(f"✅ Test Accuracy: {test_acc:.4f}")
        
        # 6. Visualizar resultados
        plot_training_history(history)
        
        return model, history
    
    elif dataset_type == "cifar10":
        # 1. Carregar dados
        (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
        train_images = train_images.astype("float32") / 255
        test_images = test_images.astype("float32") / 255
        
        # 2. Criar modelo
        model = create_cifar_cnn()
        model.compile(optimizer="rmsprop",
                     loss="sparse_categorical_crossentropy",
                     metrics=["accuracy"])
        
        # 3. Callbacks
        callbacks = create_basic_callbacks("cifar10_model")
        
        # 4. Treinar
        print("🚀 Iniciando treinamento CIFAR-10...")
        history = model.fit(train_images, train_labels,
                           epochs=20, batch_size=64,
                           validation_split=0.2,
                           callbacks=callbacks)
        
        # 5. Avaliar
        test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
        print(f"✅ Test Accuracy: {test_acc:.4f}")
        
        # 6. Visualizar resultados
        plot_training_history(history)
        
        return model, history

# Exemplo de uso:
# model, history = complete_training_workflow("mnist")
# model, history = complete_training_workflow("cifar10")

In [ ]:
## 📋 8. DICAS RÁPIDAS PARA PROVA

### ⚡ Comandos Essenciais
```python
# 1. Verificar se GPU está disponível
print("GPU disponível:", len(tf.config.list_physical_devices('GPU')) > 0)

# 2. Resumo do modelo
model.summary()

# 3. Salvar e carregar modelo
model.save("meu_modelo.keras")
model = keras.models.load_model("meu_modelo.keras")

# 4. Fazer predição em uma imagem
prediction = model.predict(image_batch)
predicted_class = np.argmax(prediction, axis=1)

# 5. Verificar overfitting
# - Training accuracy >> Validation accuracy = Overfitting
# - Solução: Dropout, Data Augmentation, Early Stopping
```

### 🎯 Checklist para Prova
- [ ] Dados carregados e pré-processados corretamente
- [ ] Modelo compilado com optimizer, loss e metrics apropriados
- [ ] Callbacks configurados (ModelCheckpoint, EarlyStopping)
- [ ] Data augmentation aplicado se necessário
- [ ] Validação durante treinamento
- [ ] Avaliação no conjunto de teste
- [ ] Visualização dos resultados

### 🔧 Troubleshooting Comum
```python
# Erro de shape: verificar dimensões dos dados
print("Shape dos dados:", train_images.shape)
print("Shape esperado pelo modelo:", model.input_shape)

# Erro de memória: reduzir batch_size
model.fit(train_images, train_labels, batch_size=16)  # em vez de 32

# Overfitting: adicionar regularização
x = layers.Dropout(0.5)(x)
x = layers.BatchNormalization()(x)
```

In [ ]:
## 🎉 9. EXEMPLO FINAL - EXECUÇÃO RÁPIDA

```python
# Exemplo completo para executar rapidamente na prova
def quick_cnn_example():
    # 1. Carregar MNIST
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    train_images = train_images.reshape(60000, 28, 28, 1).astype("float32") / 255
    test_images = test_images.reshape(10000, 28, 28, 1).astype("float32") / 255
    
    # 2. Criar modelo simples
    model = create_mnist_cnn()
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    # 3. Treinar
    history = model.fit(train_images, train_labels, epochs=5, batch_size=128, validation_split=0.2)
    
    # 4. Avaliar
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    return model, history

# Executar: model, history = quick_cnn_example()
```

### 🏆 BOA SORTE NA PROVA! 🏆